In [1]:
import rasterio
import numpy as np
import os

def create_mask(reference_image_path, nodata_value=-9999):
    with rasterio.open(reference_image_path) as ref_ds:
        reference_image = ref_ds.read(1)
        mask = (reference_image != nodata_value) & (~np.isnan(reference_image))
        transform = ref_ds.transform

    return mask, transform

def extract_data_from_folder(folder_path, mask, transform, output_folder, nodata_value=np.nan):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tif") or file_name.endswith(".tiff"):
            file_path = os.path.join(folder_path, file_name)

            with rasterio.open(file_path) as current_ds:
                current_image = current_ds.read(1)
                
                # Create a masked array with NaN values in masked areas
                masked_data = np.where(mask, current_image, np.nan)
                
                # Set the nodata value in the output
                masked_data[np.isnan(masked_data)] = nodata_value
                masked_data[masked_data>500000] = np.nan
                
                output_file_path = os.path.join(output_folder, file_name)

                # Create the output TIF file with the correct shape
                profile = current_ds.profile
                profile.update(count=1)  # Update the number of bands to 1
                profile.update(nodata=nodata_value)  # Set nodata value
                with rasterio.open(output_file_path, 'w', **profile) as dst:
                    dst.write(masked_data, 1)  # Write the masked_data directly

# Example usage
folder_path = r"/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata"
reference_image_path = r"/home/komlavi/CWA_model/SRO_monthly_data_extraction/ref_img/AFR_Incremental_ET_M_ETincr_yearly_avg_N.tif"
output_folder = r"/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/New_data"

# Create mask from the reference image based on non-NaN values
mask, transform = create_mask(reference_image_path)

# Extract data using the mask and save to the output folder
extract_data_from_folder(folder_path, mask, transform, output_folder)
